In [ ]:
# ChatGPT-4
import pulp

# 定义问题
prob = pulp.LpProblem("MIP", pulp.LpMaximize)

# 定义变量，注意这里x1和x2都是整数
x1 = pulp.LpVariable("x1", lowBound=0, cat='Integer')
x2 = pulp.LpVariable("x2", lowBound=0, cat='Integer')

# 目标函数
prob += x2

# 约束条件
prob += 3*x1 + 2*x2 <= 6
prob += -3*x1 + 2*x2 <= 0

# 解决问题
prob.solve()



# 割平面法的迭代过程
# 初始化
cuts_added = True
while cuts_added:
    cuts_added = False

    # 检查所有变量是否为整数
    for v in prob.variables():
        if v.cat == 'Integer':
            if not v.varValue.is_integer():
                # 为不满足整数条件的变量添加割平面
                floor_val = int(v.varValue)
                prob += v >= floor_val + 1
                cuts_added = True

    if cuts_added:
        # 重新解决问题
        prob.solve()
        print("Status:", pulp.LpStatus[prob.status])
        print("Optimal value with cuts: x1 =",
              pulp.value(x1), ", x2 =", pulp.value(x2))
        
# 输出结果
print("Status:", pulp.LpStatus[prob.status])
print("Optimal value: x1 =", pulp.value(x1), ", x2 =", pulp.value(x2))

In [ ]:
# ChatGPT-3.5
import pulp

# 创建混合整数线性规划问题
problem = pulp.LpProblem("IntegerProgramming", pulp.LpMaximize)

# 定义变量
x1 = pulp.LpVariable("x1", lowBound=0, cat='Integer')
x2 = pulp.LpVariable("x2", lowBound=0, cat='Integer')

# 定义目标函数
problem += x2

# 添加约束条件
problem += 3 * x1 + 2 * x2 <= 6
problem += -3 * x1 + 2 * x2 <= 0

# 解决问题
problem.solve()

# 输出结果
print("Status:", pulp.LpStatus[problem.status])
print("x1 =", x1.varValue)
print("x2 =", x2.varValue)
print("Objective =", pulp.value(problem.objective))

In [ ]:
# Github Copilot
def cut_plane_method():
    # 初始解
    x1, x2 = 0, 0
    while True:
        # 求解线性规划问题
        x1_new = max(0, min(2, 6 - 2 * x2))
        x2_new = max(0, min(3, (6 - 3 * x1) / 2))

        # 检查是否找到整数解
        if x1_new == round(x1_new) and x2_new == round(x2_new):
            return x1_new, x2_new

        # 更新解
        x1, x2 = x1_new, x2_new


# 调用函数求解问题
x1, x2 = cut_plane_method()
print("x1 =", x1)
print("x2 =", x2)

In [ ]:
# 文心一言
from scipy.optimize import linprog
def cutting_plane(c, A, b, x0, r=1e-6):
    # c: 目标函数的系数
    # A: 约束条件的系数矩阵
    # b: 约束条件的右侧向量
    # x0: 初始解
    # r: 精度
    while True:
        # 求解线性规划问题
        result = linprog(c, A_ub=A, b_ub=b, bounds=(
            0, None), method='highs', x0=x0)
        if not result.success:
            print("求解失败")
            return None
        x = result.x
        # 判断是否满足割平面条件
        if all(abs(A @ x - b) <= r):
            return x
        # 找到割平面
        i = max(range(len(A)), key=lambda i: abs(A[i] @ x - b[i]))
        # 将割平面加入到约束条件中
        A = [A[j] for j in range(len(A)) if j != i]
        b = [b[j] for j in range(len(b)) if j != i]
        

# 目标函数
def objective(x): return x[1]


# 约束条件
constraints = [
    lambda x: 3*x[0] + 2*x[1] - 6 <= 0,  # 3x_1 + 2x_2 <= 6
    lambda x: -3*x[0] + 2*x[1] - 0 <= 0  # -3x_1 + 2x_2 <= 0
]


def solve_linear_programming(c, A_ub, b_ub, bounds, method='highs'):
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method=method)
    return result.x if result.success else None


# 初始解：x1=0, x2=0（因为x_i>=0）
x0 = [0, 0]

# 调用割平面法求解问题
solution = cutting_plane(objective, constraints, x0=x0)
if solution is not None:
    print("最优解：", solution)
else:
    print("无法求解")